# `Sync version`

In [1]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama

prompt = ChatPromptTemplate.from_template("Write a comma-separated list of 5 animals similar to: {animal}")
model = ChatOllama(model="mistral", temperature=0.0)
chain = prompt | model | StrOutputParser()

In [2]:
chain.stream({"animal": "bear"})

<generator object RunnableSequence.stream at 0x10ff686d0>

In [4]:
chain.invoke({"animal": "bear"})

' 1. Elephant, known for their large size and trunk\n\n2. Rhinoceros, characterized by their thick skin and horn\n\n3. Walrus, living primarily in the Arctic with a distinctive tusk and blubber layer\n\n4. Orangutan, a primate native to Indonesia and Malaysia with long arms and a reddish-brown fur\n\n5. Hippopotamus, semi-aquatic mammal found in sub-Saharan Africa with large size and powerful jaws.'

In [3]:
for chunk in chain.stream({"animal": "bear"}):
    print(chunk, end="", flush=True)

 1. Elephant, with its large size and powerful build
2. Rhinoceros, known for their thick skin and distinctive horn
3. Hippopotamus, semi-aquatic herbivores with large bodies
4. Walrus, marine mammals with tusks and a thick layer of blubber
5. Orangutan, highly intelligent primates native to Southeast Asia with shaggy red fur.

In [5]:
from typing import Iterator, List

def split_into_list(input: Iterator[str]) -> Iterator[List[str]]:
    """
    This is a custom parser that splits an iterator of llm tokens
    into a list of strings separated by commas
    """

    buffer = ""
    for chunk in input:
        buffer += chunk
        while "," in buffer:
            comma_index = buffer.index(",")
            yield [buffer[:comma_index].strip()]
            buffer = buffer[comma_index + 1 :]
    yield [buffer.strip()]

In [6]:
list_chain = chain | split_into_list

In [7]:
for chunk in list_chain.stream({"animal": "bear"}):
    print(chunk, flush=True)

['1. Elephant']
['with its large size and powerful build\n2. Rhinoceros']
['known for their thick skin and distinctive horn\n3. Hippopotamus']
['semi-aquatic herbivores with large bodies\n4. Walrus']
['marine mammals with tusks and a thick layer of blubber\n5. Orangutan']
['highly intelligent primates native to Southeast Asia with shaggy red fur.']


In [8]:
list_chain.invoke({"animal": "bear"})

['1. Elephant',
 'known for their large size and trunk\n\n2. Rhinoceros',
 'characterized by their thick skin and horn\n\n3. Walrus',
 'living primarily in the Arctic with a distinctive tusk and blubber layer\n\n4. Orangutan',
 'a primate native to Indonesia and Malaysia with long arms and a reddish-brown fur\n\n5. Hippopotamus',
 'semi-aquatic mammal found in sub-Saharan Africa with large size and powerful jaws.']

# `Async version`

In [10]:
from typing import AsyncIterator


async def asplit_into_list(input: AsyncIterator[str],) -> AsyncIterator[List[str]]:
    """
    This is an async custom parser that splits an iterator
    of llm tokens into a list of strings separated by commas
    """

    buffer = ""
    async for (chunk) in input:
        buffer += chunk
        while "," in buffer:
            comma_index = buffer.index(",")
            yield [buffer[:comma_index].strip()]
            buffer = buffer[comma_index + 1 :]
    yield [buffer.strip()]


list_chain = chain | asplit_into_list

In [11]:
async for chunk in list_chain.astream({"animal": "bear"}):
    print(chunk, flush=True)

['1. Elephant']
['with its large size and powerful build\n2. Rhinoceros']
['known for their thick skin and distinctive horn\n3. Hippopotamus']
['semi-aquatic herbivores with large bodies\n4. Walrus']
['marine mammals with tusks and a thick layer of blubber\n5. Orangutan']
['highly intelligent primates native to Southeast Asia with shaggy red fur.']


In [12]:
await list_chain.ainvoke({"animal": "bear"})

['1. Elephant',
 'known for their large size and trunk\n\n2. Rhinoceros',
 'characterized by their thick skin and horn\n\n3. Walrus',
 'living primarily in the Arctic with a distinctive tusk and blubber layer\n\n4. Orangutan',
 'a primate native to Indonesia and Malaysia with long arms and a reddish-brown fur\n\n5. Hippopotamus',
 'semi-aquatic mammal found in sub-Saharan Africa with large size and powerful jaws.']